<a href="https://colab.research.google.com/github/shaaguunz/begin-ml/blob/master/styleGAN2_yuotube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
from google.colab import drive
drive.mount('/content/drive',force_remount= True)

TensorFlow 1.x selected.
Mounted at /content/drive


In [2]:
!git clone https://github.com/NVlabs/stylegan2.git

Cloning into 'stylegan2'...
remote: Enumerating objects: 122, done.
remote: Total 122 (delta 0), reused 0 (delta 0), pack-reused 122
Receiving objects: 100% (122/122), 590.23 KiB | 1.10 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [3]:
#runnnig from command line

!python /content/stylegan2/run_generator.py generate-images \
    --network=gdrive:networks/stylegan2-ffhq-config-f.pkl \
  --seeds=6600-6625 --truncation-psi=0.5

Local submit - run_dir: results/00000-generate-images
dnnlib: Running run_generator.generate_images() on localhost...
Loading networks from "gdrive:networks/stylegan2-ffhq-config-f.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Compiling... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Compiling... Loading... Done.
Generating image for seed 6600 (0/26) ...
Generating image for seed 6601 (1/26) ...
Generating image for seed 6602 (2/26) ...
Generating image for seed 6603 (3/26) ...
Generating image for seed 6604 (4/26) ...
Generating image for seed 6605 (5/26) ...
Generating image for seed 6606 (6/26) ...
Generating image for seed 6607 (7/26) ...
Generating image for seed 6608 (8/26) ...
Generating image for seed 6609 (9/26) ...
Generating image for seed 6610 (10/26) ...
Generating image for seed 6611 (11/26) ...
Generating image for seed 6612 (12/26) ...
Generating image for seed 6613 (13/26) ...
Generating image for seed 661

In [4]:
#now to code
import sys
sys.path.insert(0,'/content/stylegan2')


import dnnlib

In [40]:
import argparse
import numpy as np
import PIL.Image
import dnnlib.tflib as tflib
import re
import sys

import pretrained_networks


def expand_seed(seeds,vector_size):
  result =[]

  for seed in seeds:  # takes a seed and expands it into 512 
    rnd = np.random.RandomState(seed)
    result.append(rnd.randn(1,vector_size))
  return result

def generate_images(Gs, seeds, truncation_psi):  # from nvidia site Gs  network truncation_psi trick used by stylegan2
  noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]  #to add noise 


  Gs_kwargs = dnnlib.EasyDict()
  Gs_kwargs.output_transform= dict(func= tflib.convert_images_to_uint8,nchw_to_nhwc= True)
  Gs_kwargs.randomize_noise = False
  if truncation_psi is not None:
    Gs_kwargs.truncation_psi = truncation_psi

  for seed_idx, seed in enumerate(seeds):
    print('Generating images for seed %d/%d   ...'%(seed_idx, len(seeds)))
    rnd = np.random.RandomState()
    tflib.set_vars({var:rnd.randn(*var.shape.as_list()) for var in noise_vars})
    images= Gs.run(seed,None, **Gs_kwargs)
    path = f'/content/drive/My Drive/projects/stylegan2/images{seed_idx}.png'
    PIL.Image.fromarray(images[0],'RGB').save(path)



    

In [41]:
sc= dnnlib.SubmitConfig()  #loading pre trained network 
sc_num_gpus =1
sc.submit_target= dnnlib.SubmitTarget.LOCAL
sc.local.do_not_copy_source_files= True
sc.run_dir_root= '/content/drive/My Drive/projects/stylegan2'
sc.run_desc= 'generate-images'
network_pkl ='gdrive:networks/stylegan2-ffhq-config-f.pkl'

print('loading network')
_G,_D,Gs = pretrained_networks.load_networks(network_pkl)
vector_size= Gs.input_shape[1:][0]
seeds= expand_seed(range(7000,7030), vector_size)
generate_images(Gs,seeds,truncation_psi=0.5)


loading network
Generating images for seed 0/30   ...
Generating images for seed 1/30   ...
Generating images for seed 2/30   ...
Generating images for seed 3/30   ...
Generating images for seed 4/30   ...
Generating images for seed 5/30   ...
Generating images for seed 6/30   ...
Generating images for seed 7/30   ...
Generating images for seed 8/30   ...
Generating images for seed 9/30   ...
Generating images for seed 10/30   ...
Generating images for seed 11/30   ...
Generating images for seed 12/30   ...
Generating images for seed 13/30   ...
Generating images for seed 14/30   ...
Generating images for seed 15/30   ...
Generating images for seed 16/30   ...
Generating images for seed 17/30   ...
Generating images for seed 18/30   ...
Generating images for seed 19/30   ...
Generating images for seed 20/30   ...
Generating images for seed 21/30   ...
Generating images for seed 22/30   ...
Generating images for seed 23/30   ...
Generating images for seed 24/30   ...
Generating images f

In [43]:

sc = dnnlib.SubmitConfig()
sc.num_gpus = 1
sc.submit_target = dnnlib.SubmitTarget.LOCAL
sc.local.do_not_copy_source_files = True
sc.run_dir_root = "/content/drive/My Drive/projects/stylegan2"
sc.run_desc = 'generate-images'
network_pkl = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'

print('Loading networks from "%s"...' % network_pkl)
_G, _D, Gs = pretrained_networks.load_networks(network_pkl)
vector_size = Gs.input_shape[1:][0]

seeds = expand_seed( [7000+1,7000+9], vector_size)
#generate_images(Gs, seeds,truncation_psi=0.5)
print(seeds[0].shape)

Loading networks from "gdrive:networks/stylegan2-ffhq-config-f.pkl"...
(1, 512)


In [46]:
steps = 300
diff = seeds[1]-  seeds[0]
step = diff/steps
current= seeds[0].copy()

seeds2 = []
for i in range(steps):
  seeds2.append(current)
  current = current+step
generate_images(Gs,seeds2,truncation_psi=0.5)

Generating images for seed 0/300   ...
Generating images for seed 1/300   ...
Generating images for seed 2/300   ...
Generating images for seed 3/300   ...
Generating images for seed 4/300   ...
Generating images for seed 5/300   ...
Generating images for seed 6/300   ...
Generating images for seed 7/300   ...
Generating images for seed 8/300   ...
Generating images for seed 9/300   ...
Generating images for seed 10/300   ...
Generating images for seed 11/300   ...
Generating images for seed 12/300   ...
Generating images for seed 13/300   ...
Generating images for seed 14/300   ...
Generating images for seed 15/300   ...
Generating images for seed 16/300   ...
Generating images for seed 17/300   ...
Generating images for seed 18/300   ...
Generating images for seed 19/300   ...
Generating images for seed 20/300   ...
Generating images for seed 21/300   ...
Generating images for seed 22/300   ...
Generating images for seed 23/300   ...
Generating images for seed 24/300   ...
Generating

In [52]:
!ffmpeg-r 30 -i image%d.png -vcodec mpeg4 -y movie.mp4

/bin/bash: ffmpeg-r: command not found


In [49]:
pip install ffmpeg

  Created wheel for ffmpeg: filename=ffmpeg-1.4-cp36-none-any.whl size=6083 sha256=6fc68ade34e1986ee5d8217e07d8b842dbcc84562ae24747d1783398c79e3953
  Stored in directory: /root/.cache/pip/wheels/b6/68/c3/a05a35f647ba871e5572b9bbfc0b95fd1c6637a2219f959e7a
Successfully built ffmpeg


In [53]:
image

NameError: ignored